In [2]:
from pyspark import SparkContext, SparkConf
from scipy.io import loadmat
import numpy as np
from pyspark.sql.types import Row
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SQLContext
from pyspark.ml.linalg import Vectors
import os
import time
from sklearn.metrics import roc_curve, auc
from pyspark.ml import Pipeline
import gcsfs
import json




subjects = ['Patient_8']   

num_nodes = 2
appName = 'seizure_detection'
conf = SparkConf().setAppName(appName).setMaster('local')
conf = (conf.setMaster('local[*]')
        .set("spark.executor.instances", str(2 * num_nodes))
        .set('spark.executor.memory', '15G')
        .set('spark.driver.memory', '15G')
        .set('spark.driver.maxResultSize', '15G'))
try:
    sc.stop()
except:
    pass
sc = SparkContext(conf = conf)

sqlContext = SQLContext(sc)
    
gs_dir = "gs://seizure_detection_data/notebooks/seizure_detection_spark_gcp"
json_str_rdd = sc.textFile(gs_dir + '/SETTINGS.json')
json_str = ''.join(json_str_rdd.collect())
settings = json.loads(json_str)

proj_name = settings['gcp-project-name']
proj_dir = settings['gcp-bucket-project-dir']

fs = gcsfs.GCSFileSystem(project=proj_name)
fopen = fs.open(proj_dir + '/spark_data_io.py')
exec(fopen.read())
fopen.close()
fopen = fs.open(proj_dir + '/spark_transform.py')
exec(fopen.read())
fopen.close()
fopen = fs.open(proj_dir + '/spark_processing.py')
exec(fopen.read())
fopen.close()


trained_models = train_model(gs_dir, subjects, sc, fs, num_nodes)





--- Patient_8: Data Loading 384.00960326194763 seconds ---
1890
--- Patient_8: Data Transformation 25.397717475891113 seconds ---
--- Patient_8: Model Training 372.35589838027954 seconds ---
--- Patient_8: Saving Trained Model ---
Trained classifier saved
Trained classifier loaded


RandomForestClassificationModel (uid=RandomForestClassifier_4cc288e960d73feec692) with 3000 trees